# Kaggle - nivel 1

## Selección de los atributos

seguimos con la competición Titanic Challenge visto en el punto 1

- v1.1_042020
- author and contact: [marcusRB](https://www.linkedin.com/in/marcusrb)
- [Kaggle - Titanic challenge](https://www.kaggle.com/c/titanic/)

#### NOTA

Este paso es importante para la selección de las dimensiones con mejor capacidad de predicción. Se realizarán tareas de correlación, o reducción de dimensionalidad, basada en distancias, estudio de la varianza etc.

***

## Import Libraries

In [4]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
from scipy.stats import norm
import matplotlib.pyplot as plt
%matplotlib inline

# data mining
#from sklearn.impute import KNNImputer, MissingIndicator, SimpleImputer
from sklearn import impute
from sklearn_pandas import categorical_imputer, CategoricalImputer
from sklearn.pipeline import make_pipeline, make_union
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler


# machine learning
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier

## scikit modeling libraries
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier,
                             GradientBoostingClassifier, ExtraTreesClassifier,
                             VotingClassifier)

from sklearn.model_selection import (GridSearchCV, cross_val_score, cross_val_predict,
                                     StratifiedKFold, learning_curve)

## Load metrics for predictive modeling
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE, rfe
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import mean_absolute_error, mean_squared_error


# Save the model
import pickle
import joblib


## Warnings and other tools
import itertools
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.rfe module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection.rfe. Anything that cannot be imported from sklearn.feature_selection.rfe is now part of the private API.
  warnings.warn(message, FutureWarning)


***

## Load dataset

Kaggle we provide two datasets: train and test in csv extension. So, we check and analyze only train file.

In [5]:
# Carga de los ficheros anteriores ya preprocesados - cambiar la ruta según vuestro directorio o raíz
train_titanic = pd.read_csv('train_eda.csv')
test_titanic = pd.read_csv('test_eda.csv')

In [6]:
train_titanic.head(10)

,Survived,Pclass,Sex,SibSp,Parch,Embarked,FamilySize,IsAlone,FareGroup,InCabin,Deck,RoomGroup,AgeGroup,Title
0,0,3,True,1,0,S,2,0,A,False,N,r1,Student,Mr
1,1,1,False,1,0,C,2,0,D,True,C,r3,Adult,Mrs
2,1,3,False,0,0,S,1,1,B,False,N,r1,Young Adult,Miss
3,1,1,False,1,0,S,2,0,D,True,C,r4,Young Adult,Mrs
4,0,3,True,0,0,S,1,1,B,False,N,r1,Young Adult,Mr
5,0,3,True,0,0,Q,1,1,B,False,N,r1,Unknown,Mr
6,0,1,True,0,0,S,1,1,D,True,E,r1,Adult,Mr
7,0,3,True,3,1,S,5,0,C,False,N,r1,Baby,Master
8,1,3,False,0,2,S,3,0,B,False,N,r1,Young Adult,Mrs
9,1,2,False,1,0,C,2,0,C,False,N,r1,Teenager,Mrs


In [7]:
test_titanic.head(10)

,PassengerId,Pclass,Sex,SibSp,Parch,Embarked,FamilySize,IsAlone,FareGroup,InCabin,Deck,RoomGroup,AgeGroup,Title
0,892,3,True,0,0,Q,1,1,A,False,N,r1,Student,Mr
1,893,3,False,1,0,S,2,0,A,False,N,r1,Adult,Mrs
2,894,2,True,0,0,Q,1,1,B,False,N,r1,Young Adult,Mr
3,895,3,True,0,0,S,1,1,B,False,N,r1,Young Adult,Mr
4,896,3,False,1,1,S,3,0,B,False,N,r1,Young Adult,Mrs
5,897,3,True,0,0,S,1,1,B,False,N,r1,Unknown,Mr
6,898,3,False,0,0,Q,1,1,A,False,N,r1,Adult,Miss
7,899,2,True,1,1,S,3,0,C,False,N,r1,Baby,Mr
8,900,3,False,0,0,C,1,1,A,False,N,r1,Young Adult,Mrs
9,901,3,True,2,0,S,3,0,C,False,N,r1,Teenager,Mr


* * *

### Training and Validation dataset

In [8]:
PassengerId = test_titanic['PassengerId']
X = train_titanic
Y = test_titanic.drop('PassengerId', axis=1).copy()

In [9]:
PassengerId

0       892
1       893
2       894
3       895
4       896
       ... 
413    1305
414    1306
415    1307
416    1308
417    1309
Name: PassengerId, Length: 418, dtype: int64

In [10]:
Y

,Pclass,Sex,SibSp,Parch,Embarked,FamilySize,IsAlone,FareGroup,InCabin,Deck,RoomGroup,AgeGroup,Title
0,3,True,0,0,Q,1,1,A,False,N,r1,Student,Mr
1,3,False,1,0,S,2,0,A,False,N,r1,Adult,Mrs
2,2,True,0,0,Q,1,1,B,False,N,r1,Young Adult,Mr
3,3,True,0,0,S,1,1,B,False,N,r1,Young Adult,Mr
4,3,False,1,1,S,3,0,B,False,N,r1,Young Adult,Mrs
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,3,True,0,0,S,1,1,B,False,N,r1,Unknown,Mr
414,1,False,0,0,C,1,1,D,True,C,r4,Adult,Rare
415,3,True,0,0,S,1,1,A,False,N,r1,Unknown,Mr
416,3,True,0,0,S,1,1,B,False,N,r1,Young Adult,Mr


* * *

## Scaling with StandardScaler

Utilizamos la librería `StandarScaler` para poner en la misma escala las dimensiones siguientes.
`SibSp`, `Parch`, `FamilySize`

In [15]:
# combinamos train and test en un solo dataframe
dataset = [X, Y]

In [16]:
col_Z = ['SibSp', 'Parch', 'FamilySize']

In [17]:
features = dataset[0][col_Z]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)

In [18]:
dataset[0][col_Z] = features
dataset[0].head(10)

,Survived,Pclass,Sex,SibSp,Parch,Embarked,FamilySize,IsAlone,FareGroup,InCabin,Deck,RoomGroup,AgeGroup,Title
0,0,3,True,0.432793,-0.473674,S,0.059160,0,A,False,N,r1,Student,Mr
1,1,1,False,0.432793,-0.473674,C,0.059160,0,D,True,C,r3,Adult,Mrs
2,1,3,False,-0.474545,-0.473674,S,-0.560975,1,B,False,N,r1,Young Adult,Miss
3,1,1,False,0.432793,-0.473674,S,0.059160,0,D,True,C,r4,Young Adult,Mrs
4,0,3,True,-0.474545,-0.473674,S,-0.560975,1,B,False,N,r1,Young Adult,Mr
5,0,3,True,-0.474545,-0.473674,Q,-0.560975,1,B,False,N,r1,Unknown,Mr
6,0,1,True,-0.474545,-0.473674,S,-0.560975,1,D,True,E,r1,Adult,Mr
7,0,3,True,2.247470,0.767630,S,1.919564,0,C,False,N,r1,Baby,Master
8,1,3,False,-0.474545,2.008933,S,0.679295,0,B,False,N,r1,Young Adult,Mrs
9,1,2,False,0.432793,-0.473674,C,0.059160,0,C,False,N,r1,Teenager,Mrs


In [19]:
features = dataset[1][col_Z]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)

In [20]:
dataset[1][col_Z] = features
dataset[1].head(10)

,Pclass,Sex,SibSp,Parch,Embarked,FamilySize,IsAlone,FareGroup,InCabin,Deck,RoomGroup,AgeGroup,Title
0,3,True,-0.499470,-0.400248,Q,-0.553443,1,A,False,N,r1,Student,Mr
1,3,False,0.616992,-0.400248,S,0.105643,0,A,False,N,r1,Adult,Mrs
2,2,True,-0.499470,-0.400248,Q,-0.553443,1,B,False,N,r1,Young Adult,Mr
3,3,True,-0.499470,-0.400248,S,-0.553443,1,B,False,N,r1,Young Adult,Mr
4,3,False,0.616992,0.619896,S,0.764728,0,B,False,N,r1,Young Adult,Mrs
5,3,True,-0.499470,-0.400248,S,-0.553443,1,B,False,N,r1,Unknown,Mr
6,3,False,-0.499470,-0.400248,Q,-0.553443,1,A,False,N,r1,Adult,Miss
7,2,True,0.616992,0.619896,S,0.764728,0,C,False,N,r1,Baby,Mr
8,3,False,-0.499470,-0.400248,C,-0.553443,1,A,False,N,r1,Young Adult,Mrs
9,3,True,1.733455,-0.400248,S,0.764728,0,C,False,N,r1,Teenager,Mr


## Binarizing categorical variables

De cada variable categórica haremos `m` variables binarias, donde `m` es el numero de categorías de la variable.

In [21]:
cols = ['Pclass', 'Embarked', 'FareGroup', 'AgeGroup', 'Title', 'Deck', 'RoomGroup']
titanic_categorical = dataset[0][cols]
titanic_categorical = pd.concat([pd.get_dummies(titanic_categorical[col], prefix=col) for col in titanic_categorical], axis=1)
titanic_categorical.head()
dataset[0] = pd.concat([dataset[0][dataset[0].columns[~dataset[0].columns.isin(cols)]], titanic_categorical], axis=1)
dataset[0].head()

,Survived,Sex,SibSp,Parch,FamilySize,IsAlone,InCabin,Pclass_1,Pclass_2,Pclass_3,...,Deck_D,Deck_E,Deck_F,Deck_G,Deck_N,Deck_T,RoomGroup_r1,RoomGroup_r2,RoomGroup_r3,RoomGroup_r4
0,0,True,0.432793,-0.473674,0.059160,0,False,0,0,1,...,0,0,0,0,1,0,1,0,0,0
1,1,False,0.432793,-0.473674,0.059160,0,True,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1,False,-0.474545,-0.473674,-0.560975,1,False,0,0,1,...,0,0,0,0,1,0,1,0,0,0
3,1,False,0.432793,-0.473674,0.059160,0,True,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,True,-0.474545,-0.473674,-0.560975,1,False,0,0,1,...,0,0,0,0,1,0,1,0,0,0


In [22]:
titanic_categorical = dataset[1][cols]
titanic_categorical = pd.concat([pd.get_dummies(titanic_categorical[col], prefix=col) for col in titanic_categorical], axis=1)
dataset[1] = pd.concat([dataset[1][dataset[1].columns[~dataset[1].columns.isin(cols)]], titanic_categorical], axis=1)
dataset[1].head()

,Sex,SibSp,Parch,FamilySize,IsAlone,InCabin,Pclass_1,Pclass_2,Pclass_3,Embarked_C,...,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_N,RoomGroup_r1,RoomGroup_r2,RoomGroup_r3,RoomGroup_r4
0,True,-0.499470,-0.400248,-0.553443,1,False,0,0,1,0,...,0,0,0,0,0,1,1,0,0,0
1,False,0.616992,-0.400248,0.105643,0,False,0,0,1,0,...,0,0,0,0,0,1,1,0,0,0
2,True,-0.499470,-0.400248,-0.553443,1,False,0,1,0,0,...,0,0,0,0,0,1,1,0,0,0
3,True,-0.499470,-0.400248,-0.553443,1,False,0,0,1,0,...,0,0,0,0,0,1,1,0,0,0
4,False,0.616992,0.619896,0.764728,0,False,0,0,1,0,...,0,0,0,0,0,1,1,0,0,0


In [23]:
# We remove Deck_T from dataset0
dataset[0].columns.tolist()

['Survived',
 'Sex',
 'SibSp',
 'Parch',
 'FamilySize',
 'IsAlone',
 'InCabin',
 'Pclass_1',
 'Pclass_2',
 'Pclass_3',
 'Embarked_C',
 'Embarked_Q',
 'Embarked_S',
 'FareGroup_A',
 'FareGroup_B',
 'FareGroup_C',
 'FareGroup_D',
 'AgeGroup_Adult',
 'AgeGroup_Baby',
 'AgeGroup_Child',
 'AgeGroup_Senior',
 'AgeGroup_Student',
 'AgeGroup_Teenager',
 'AgeGroup_Unknown',
 'AgeGroup_Young Adult',
 'Title_Master',
 'Title_Miss',
 'Title_Mr',
 'Title_Mrs',
 'Title_Rare',
 'Deck_A',
 'Deck_B',
 'Deck_C',
 'Deck_D',
 'Deck_E',
 'Deck_F',
 'Deck_G',
 'Deck_N',
 'Deck_T',
 'RoomGroup_r1',
 'RoomGroup_r2',
 'RoomGroup_r3',
 'RoomGroup_r4']

In [24]:
dataset[1].columns.tolist()

['Sex',
 'SibSp',
 'Parch',
 'FamilySize',
 'IsAlone',
 'InCabin',
 'Pclass_1',
 'Pclass_2',
 'Pclass_3',
 'Embarked_C',
 'Embarked_Q',
 'Embarked_S',
 'FareGroup_A',
 'FareGroup_B',
 'FareGroup_C',
 'FareGroup_D',
 'AgeGroup_Adult',
 'AgeGroup_Baby',
 'AgeGroup_Child',
 'AgeGroup_Senior',
 'AgeGroup_Student',
 'AgeGroup_Teenager',
 'AgeGroup_Unknown',
 'AgeGroup_Young Adult',
 'Title_Master',
 'Title_Miss',
 'Title_Mr',
 'Title_Mrs',
 'Title_Rare',
 'Deck_A',
 'Deck_B',
 'Deck_C',
 'Deck_D',
 'Deck_E',
 'Deck_F',
 'Deck_G',
 'Deck_N',
 'RoomGroup_r1',
 'RoomGroup_r2',
 'RoomGroup_r3',
 'RoomGroup_r4']

In [25]:
# Eliminamos `Deck_T` por estar incluida en testing
dataset[0] = dataset[0].drop('Deck_T', axis=1).copy()

In [26]:
dataset[0]

,Survived,Sex,SibSp,Parch,FamilySize,IsAlone,InCabin,Pclass_1,Pclass_2,Pclass_3,...,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_N,RoomGroup_r1,RoomGroup_r2,RoomGroup_r3,RoomGroup_r4
0,0,True,0.432793,-0.473674,0.059160,0,False,0,0,1,...,0,0,0,0,0,1,1,0,0,0
1,1,False,0.432793,-0.473674,0.059160,0,True,1,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,False,-0.474545,-0.473674,-0.560975,1,False,0,0,1,...,0,0,0,0,0,1,1,0,0,0
3,1,False,0.432793,-0.473674,0.059160,0,True,1,0,0,...,1,0,0,0,0,0,0,0,0,1
4,0,True,-0.474545,-0.473674,-0.560975,1,False,0,0,1,...,0,0,0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,True,-0.474545,-0.473674,-0.560975,1,False,0,1,0,...,0,0,0,0,0,1,1,0,0,0
887,1,False,-0.474545,-0.473674,-0.560975,1,True,1,0,0,...,0,0,0,0,0,0,1,0,0,0
888,0,False,0.432793,2.008933,1.299429,0,False,0,0,1,...,0,0,0,0,0,1,1,0,0,0
889,1,True,-0.474545,-0.473674,-0.560975,1,True,1,0,0,...,1,0,0,0,0,0,0,0,0,1


In [27]:
dataset[0].dtypes

Survived                  int64
Sex                        bool
SibSp                   float64
Parch                   float64
FamilySize              float64
IsAlone                   int64
InCabin                    bool
Pclass_1                  uint8
Pclass_2                  uint8
Pclass_3                  uint8
Embarked_C                uint8
Embarked_Q                uint8
Embarked_S                uint8
FareGroup_A               uint8
FareGroup_B               uint8
FareGroup_C               uint8
FareGroup_D               uint8
AgeGroup_Adult            uint8
AgeGroup_Baby             uint8
AgeGroup_Child            uint8
AgeGroup_Senior           uint8
AgeGroup_Student          uint8
AgeGroup_Teenager         uint8
AgeGroup_Unknown          uint8
AgeGroup_Young Adult      uint8
Title_Master              uint8
Title_Miss                uint8
Title_Mr                  uint8
Title_Mrs                 uint8
Title_Rare                uint8
Deck_A                    uint8
Deck_B  

* * *

## Variable correlation


The `DataFrame.corr()` method displays the correlation (or relationship) between the columns. We’ll examine the correlations between the features and the target by Pearson, Kendall and Spearman

- What is Correlation?
- Test Dataset
- Covariance
- Pearson’s Correlation
- Spearman’s Correlation


In [28]:
dataset[0].dtypes

Survived                  int64
Sex                        bool
SibSp                   float64
Parch                   float64
FamilySize              float64
IsAlone                   int64
InCabin                    bool
Pclass_1                  uint8
Pclass_2                  uint8
Pclass_3                  uint8
Embarked_C                uint8
Embarked_Q                uint8
Embarked_S                uint8
FareGroup_A               uint8
FareGroup_B               uint8
FareGroup_C               uint8
FareGroup_D               uint8
AgeGroup_Adult            uint8
AgeGroup_Baby             uint8
AgeGroup_Child            uint8
AgeGroup_Senior           uint8
AgeGroup_Student          uint8
AgeGroup_Teenager         uint8
AgeGroup_Unknown          uint8
AgeGroup_Young Adult      uint8
Title_Master              uint8
Title_Miss                uint8
Title_Mr                  uint8
Title_Mrs                 uint8
Title_Rare                uint8
Deck_A                    uint8
Deck_B  

In [29]:
# Creación de la matriz de correlación
k = 16 #number of variables for heatmap
corrmat = dataset[0].corr(method='spearman')
# picking the top 15 correlated features
cols15 = corrmat.nlargest(k, 'Survived')['Survived'].index

Se pueden realizar diferentes cálculos para poder obtener un listado de columnas con mayor correlación. También, dependiendo del caso, realizar reducción de dimensionalidad como `LDA` o `PCA`

In [30]:
# Show 15 features with most correlation ratio - Pearson
corr = dataset[0].corr(method='pearson')
print (corr['Survived'].sort_values(ascending=False)[:15], '\n')
print (corr['Survived'].sort_values(ascending=False)[-5:])

Survived         1.000000
Title_Mrs        0.341994
Title_Miss       0.335636
InCabin          0.316912
Pclass_1         0.285904
FareGroup_D      0.233638
Deck_B           0.175095
Embarked_C       0.168240
Deck_D           0.150716
AgeGroup_Baby    0.150304
Deck_E           0.145321
Deck_C           0.114652
RoomGroup_r4     0.096760
Pclass_2         0.093349
RoomGroup_r3     0.089330
Name: Survived, dtype: float64 

FareGroup_A   -0.221610
Deck_N        -0.316912
Pclass_3      -0.322308
Sex           -0.543351
Title_Mr      -0.549199
Name: Survived, dtype: float64


In [31]:
# Show 15 features with most correlation ratio - Kendall
corr = dataset[0].corr(method='kendall')
print (corr['Survived'].sort_values(ascending=False)[:15], '\n')
print (corr['Survived'].sort_values(ascending=False)[-5:])

Survived         1.000000
Title_Mrs        0.341994
Title_Miss       0.335636
InCabin          0.316912
Pclass_1         0.285904
FareGroup_D      0.233638
Deck_B           0.175095
Embarked_C       0.168240
FamilySize       0.154863
Deck_D           0.150716
AgeGroup_Baby    0.150304
Deck_E           0.145321
Parch            0.133933
Deck_C           0.114652
RoomGroup_r4     0.096760
Name: Survived, dtype: float64 

FareGroup_A   -0.221610
Deck_N        -0.316912
Pclass_3      -0.322308
Sex           -0.543351
Title_Mr      -0.549199
Name: Survived, dtype: float64


In [32]:
# Show 15 features with most correlation ratio - Spearman
corr = dataset[0].corr(method='spearman')
print (corr['Survived'].sort_values(ascending=False)[:15], '\n')
print (corr['Survived'].sort_values(ascending=False)[-5:])

Survived         1.000000
Title_Mrs        0.341994
Title_Miss       0.335636
InCabin          0.316912
Pclass_1         0.285904
FareGroup_D      0.233638
Deck_B           0.175095
Embarked_C       0.168240
FamilySize       0.165463
Deck_D           0.150716
AgeGroup_Baby    0.150304
Deck_E           0.145321
Parch            0.138266
Deck_C           0.114652
RoomGroup_r4     0.096760
Name: Survived, dtype: float64 

FareGroup_A   -0.221610
Deck_N        -0.316912
Pclass_3      -0.322308
Sex           -0.543351
Title_Mr      -0.549199
Name: Survived, dtype: float64


* * *
## Features selection

> Compared with the `second dataset` we try to remove features related with Sales and choose another ones.


## <a id='5.3'>5.3. Recursive Feature Elimination</a>
Recursive Feature Elimination (RFE) is based on the idea to repeatedly construct a model and choose either the best or worst performing feature, setting the feature aside and then repeating the process with the rest of the features. This process is applied until all features in the dataset are exhausted. The goal of RFE is to select features by recursively considering smaller and smaller sets of features.

In [33]:
X_feat = dataset[0].drop('Survived', axis=1).copy()
y_feat = dataset[0]['Survived']

In [34]:
X_feat

,Sex,SibSp,Parch,FamilySize,IsAlone,InCabin,Pclass_1,Pclass_2,Pclass_3,Embarked_C,...,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_N,RoomGroup_r1,RoomGroup_r2,RoomGroup_r3,RoomGroup_r4
0,True,0.432793,-0.473674,0.059160,0,False,0,0,1,0,...,0,0,0,0,0,1,1,0,0,0
1,False,0.432793,-0.473674,0.059160,0,True,1,0,0,1,...,1,0,0,0,0,0,0,0,1,0
2,False,-0.474545,-0.473674,-0.560975,1,False,0,0,1,0,...,0,0,0,0,0,1,1,0,0,0
3,False,0.432793,-0.473674,0.059160,0,True,1,0,0,0,...,1,0,0,0,0,0,0,0,0,1
4,True,-0.474545,-0.473674,-0.560975,1,False,0,0,1,0,...,0,0,0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,True,-0.474545,-0.473674,-0.560975,1,False,0,1,0,0,...,0,0,0,0,0,1,1,0,0,0
887,False,-0.474545,-0.473674,-0.560975,1,True,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
888,False,0.432793,2.008933,1.299429,0,False,0,0,1,0,...,0,0,0,0,0,1,1,0,0,0
889,True,-0.474545,-0.473674,-0.560975,1,True,1,0,0,1,...,1,0,0,0,0,0,0,0,0,1


In [35]:
features = X_feat.columns.tolist()

In [36]:
# Get k
n = 17

# Create model by Logistic Regression and calculate RMSE
lr = LogisticRegression(n_jobs=4, verbose=2)
rfe = RFE(lr, n, verbose=2, )
rfe = rfe.fit(X_feat, y_feat.values.ravel())

Fitting estimator with 41 features.


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.9s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting estimator with 40 features.


[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.4s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting estimator with 39 features.
Fitting estimator with 38 features.
Fitting estimator with 37 features.
Fitting estimator with 36 features.
Fitting estimator with 35 features.
Fitting estimator with 34 features.


...

[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concu

Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.


[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.4s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.


[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.4s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.0s finished


In [37]:
rfe.estimator_

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=4, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=2,
                   warm_start=False)

In [38]:
rfe.n_features_

17

In [39]:
# Print Support and Ranking
print(rfe.support_)
print(rfe.ranking_)

[ True False False  True False  True False False  True False False False
  True  True False False  True  True False  True False False False False
  True False  True  True  True False  True False  True  True False  True
 False False False False False]
[ 1  7 18  1  8  1  9 10  1 16 17  2  1  1 19  4  1  1 12  1 22 21 15  6
  1 11  1  1  1 23  1 14  1  1 20  1  3 24 25  5 13]


In [40]:
z=zip(features, rfe.support_, rfe.ranking_)
list(z)

[('Sex', True, 1),
 ('SibSp', False, 7),
 ('Parch', False, 18),
 ('FamilySize', True, 1),
 ('IsAlone', False, 8),
 ('InCabin', True, 1),
 ('Pclass_1', False, 9),
 ('Pclass_2', False, 10),
 ('Pclass_3', True, 1),
 ('Embarked_C', False, 16),
 ('Embarked_Q', False, 17),
 ('Embarked_S', False, 2),
 ('FareGroup_A', True, 1),
 ('FareGroup_B', True, 1),
 ('FareGroup_C', False, 19),
 ('FareGroup_D', False, 4),
 ('AgeGroup_Adult', True, 1),
 ('AgeGroup_Baby', True, 1),
 ('AgeGroup_Child', False, 12),
 ('AgeGroup_Senior', True, 1),
 ('AgeGroup_Student', False, 22),
 ('AgeGroup_Teenager', False, 21),
 ('AgeGroup_Unknown', False, 15),
 ('AgeGroup_Young Adult', False, 6),
 ('Title_Master', True, 1),
 ('Title_Miss', False, 11),
 ('Title_Mr', True, 1),
 ('Title_Mrs', True, 1),
 ('Title_Rare', True, 1),
 ('Deck_A', False, 23),
 ('Deck_B', True, 1),
 ('Deck_C', False, 14),
 ('Deck_D', True, 1),
 ('Deck_E', True, 1),
 ('Deck_F', False, 20),
 ('Deck_G', True, 1),
 ('Deck_N', False, 3),
 ('RoomGroup_r1', 

In [41]:
colsRNK10 = []
for v, s, r in zip(features, rfe.support_, rfe.ranking_):
    if r >=1 and r <=10:
        colsRNK10.append(v)

In [42]:
colsRNK15 = []
for v, s, r in zip(features, rfe.support_, rfe.ranking_):
    if r >=1 and r <=15:
        colsRNK15.append(v)

In [43]:
colsRNK20 = []
for v, s, r in zip(features, rfe.support_, rfe.ranking_):
    if r >=1 and r <=20:
        colsRNK20.append(v)

In [44]:
colsSPT = []
for v, s, r in zip(features, rfe.support_, rfe.ranking_):
    if s == True:
        colsSPT.append(v)

In [45]:
print(len(colsRNK10)) # Are the features selected by Ranking10
print(len(colsRNK15)) # Are the features selected by Ranking15
print(len(colsRNK20)) # Are the features selected by Ranking20

print(len(colsSPT)) # Are the features selected by Support

26
31
36
17


In [46]:
# Show all Features selected
print(cols15)
print("*-*"*20)
print(colsRNK10)
print("*-*"*20)
print(colsRNK15)
print("*-*"*20)
print(colsRNK20)
print("*-*"*20)
print(colsSPT)

Index(['Survived', 'Title_Mrs', 'Title_Miss', 'InCabin', 'Pclass_1',
       'FareGroup_D', 'Deck_B', 'Embarked_C', 'FamilySize', 'Deck_D',
       'AgeGroup_Baby', 'Deck_E', 'Parch', 'Deck_C', 'RoomGroup_r4',
       'Pclass_2'],
      dtype='object')
*-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-*
['Sex', 'SibSp', 'FamilySize', 'IsAlone', 'InCabin', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Embarked_S', 'FareGroup_A', 'FareGroup_B', 'FareGroup_D', 'AgeGroup_Adult', 'AgeGroup_Baby', 'AgeGroup_Senior', 'AgeGroup_Young Adult', 'Title_Master', 'Title_Mr', 'Title_Mrs', 'Title_Rare', 'Deck_B', 'Deck_D', 'Deck_E', 'Deck_G', 'Deck_N', 'RoomGroup_r3']
*-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-*
['Sex', 'SibSp', 'FamilySize', 'IsAlone', 'InCabin', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Embarked_S', 'FareGroup_A', 'FareGroup_B', 'FareGroup_D', 'AgeGroup_Adult', 'AgeGroup_Baby', 'AgeGroup_Child', 'AgeGroup_Senior', 'AgeGroup_Unknown', 'AgeGroup_Young Adult', 'Title_Master', '

In [47]:
# Eliminamos Survived 
cols15 = cols15.drop('Survived')

Una vez tengamos los datos preformateado y listo para enviarlos al modelo, guardamos los ficheros para comenzar con el primer modelo `Logistic Regression`.

## Save 2nd Feature Engineering data part

guardamos las variables de esta segunda parte

In [69]:
# Save dataset0 and dataset1 for next step: Modeling
dataset[0].to_csv('train_feat.csv', index=False)
dataset[1].to_csv('test_feat.csv', index=False)

In [ ]:
# Guardamos PassengerID que también podemos recuperar del fichero original test['PassengerId']
# PassengerId = test_titanic['PassengerId']

In [81]:
# Procederemos con guardar las columnas como Series y utilziarlas para nuestros modelos
corr_columns = [cols15, colsRNK10, colsRNK15, colsRNK20, colsSPT]
cols_feat = pd.Series(corr_columns)

In [87]:
# Guardamos el listado en formato Pickle
cols_feat.to_pickle('cols_feat.pkl')

#### Es importante poder realizar las comprobaciones para pasar al otro notebook

In [85]:
a = pd.read_pickle('cols_feat.pkl')
a

0    Index(['Title_Mrs', 'Title_Miss', 'InCabin', '...
1    [Sex, SibSp, FamilySize, IsAlone, InCabin, Pcl...
2    [Sex, SibSp, FamilySize, IsAlone, InCabin, Pcl...
3    [Sex, SibSp, Parch, FamilySize, IsAlone, InCab...
4    [Sex, FamilySize, InCabin, Pclass_3, FareGroup...
dtype: object

In [86]:
a[0]

Index(['Title_Mrs', 'Title_Miss', 'InCabin', 'Pclass_1', 'FareGroup_D',
       'Deck_B', 'Embarked_C', 'FamilySize', 'Deck_D', 'AgeGroup_Baby',
       'Deck_E', 'Parch', 'Deck_C', 'RoomGroup_r4', 'Pclass_2'],
      dtype='object')

In [88]:
b = pd.read_csv('train_feat.csv')
b.head(10)

,Survived,Sex,SibSp,Parch,FamilySize,IsAlone,InCabin,Pclass_1,Pclass_2,Pclass_3,...,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_N,RoomGroup_r1,RoomGroup_r2,RoomGroup_r3,RoomGroup_r4
0,0,True,0.432793,-0.473674,0.059160,0,False,0,0,1,...,0,0,0,0,0,1,1,0,0,0
1,1,False,0.432793,-0.473674,0.059160,0,True,1,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,False,-0.474545,-0.473674,-0.560975,1,False,0,0,1,...,0,0,0,0,0,1,1,0,0,0
3,1,False,0.432793,-0.473674,0.059160,0,True,1,0,0,...,1,0,0,0,0,0,0,0,0,1
4,0,True,-0.474545,-0.473674,-0.560975,1,False,0,0,1,...,0,0,0,0,0,1,1,0,0,0
5,0,True,-0.474545,-0.473674,-0.560975,1,False,0,0,1,...,0,0,0,0,0,1,1,0,0,0
6,0,True,-0.474545,-0.473674,-0.560975,1,True,1,0,0,...,0,0,1,0,0,0,1,0,0,0
7,0,True,2.247470,0.767630,1.919564,0,False,0,0,1,...,0,0,0,0,0,1,1,0,0,0
8,1,False,-0.474545,2.008933,0.679295,0,False,0,0,1,...,0,0,0,0,0,1,1,0,0,0
9,1,False,0.432793,-0.473674,0.059160,0,False,0,1,0,...,0,0,0,0,0,1,1,0,0,0


Perfecto! - Seguimos con el primer modelo